In [7]:
from dotenv.main import load_dotenv
import os
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
import json
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.chains import LLMChain
def output_repraser(input_string):
    json_str = input_string.strip('```json\n').rstrip('\n```').strip()
    
    # Step 2: Parse the JSON string into a dictionary
    data_dict = json.loads(json_str)
    return data_dict

class ChatModelWorker:
    def __init__(self, output_parser,temperature=0, model='gpt-4'):
        with open('api_key.txt','r') as f:
            apikey = f.read()
        self.chat_model = ChatOpenAI(openai_api_key=apikey, model_name=model, temperature=temperature)
        self.output_parser = output_parser

    def prompt_temps(self, sys_temp, human_temp, format_instructions):

        sys_msg_prompt = SystemMessagePromptTemplate.from_template(sys_temp)
        human_msg_prompt = HumanMessagePromptTemplate.from_template(human_temp)
        chat_prompt = ChatPromptTemplate(partial_variables={"format_instructions": format_instructions},
                                         messages=[sys_msg_prompt, human_msg_prompt])
        return chat_prompt


    def chain_generator(self, template, human_template):
        output_parser = self.output_parser
        format_instructions = output_parser.get_format_instructions()
        chain = LLMChain(
            llm=self.chat_model,
            prompt=self.prompt_temps(template, human_template, format_instructions)
        )
        return chain


# Direct F-B

In [ ]:

class ChatModelWorker:
    def __init__(self, output_parser,temperature=0, model='gpt-4'):
        with open('api_key.txt','r') as f:
            apikey = f.read()
        self.chat_model = ChatOpenAI(openai_api_key=apikey, model_name=model, temperature=temperature)
        self.output_parser = output_parser

    def prompt_temps(self, sys_temp, human_temp, format_instructions):

        sys_msg_prompt = SystemMessagePromptTemplate.from_template(sys_temp)
        human_msg_prompt = HumanMessagePromptTemplate.from_template(human_temp)
        chat_prompt = ChatPromptTemplate(partial_variables={"format_instructions": format_instructions},
                                         messages=[sys_msg_prompt, human_msg_prompt])
        return chat_prompt


    def chain_generator(self, template, human_template):
        output_parser = self.output_parser
        format_instructions = output_parser.get_format_instructions()
        chain = LLMChain(
            llm=self.chat_model,
            prompt=self.prompt_temps(template, human_template, format_instructions)
        )
        return chain

import json

def output_repraser(input_string):
    json_str = input_string.strip('```json\n').rstrip('\n```').strip()
    
    # Step 2: Parse the JSON string into a dictionary
    data_dict = json.loads(json_str)
    return data_dict



def qa_agent(subject, question, temp=0, model_name='gpt-4-0125-preview'):
    system_prompt = (
        "You are a professional specialized in {subject}. You need to help me answer the given question."
        "Notice that you need to solve the question step by step. Do not jump to the answer directly."
        "Your intermediate steps and thoughts are critical!"
        "\n{format_instructions}")
    human_prompt = "{question}"
        
    response_schemas = [
        ResponseSchema(name="Chain of Thought",
                       description="Provide step by step analysis. For instance, you should follow the pattern 'step 1:... \nstep 2:...'"),
        ResponseSchema(name="Backwards Chain of Thought",
                       description="Now, you use your obtained answer and performing reverse checking. In other words, you plug in the answer to the steps to verify if each step holds. For instance, you should start from the last step you proposed the pattern 'step n:... \nstep n-1:...'"),
        ResponseSchema(name="Final Answer",
                       description="Give me your original answer and your backward analysis answer")
    ]
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    success = False
    while not success:
        try:
            worker = ChatModelWorker(output_parser=output_parser,temperature=temp, model=model_name)
            chain = worker.chain_generator(system_prompt, human_prompt)
            out_put = chain.run(subject=subject,
                                question=question)

            success = True
        except:
            worker = ChatModelWorker(output_parser=output_parser,temperature=temp, model=model_name)
            chain = worker.chain_generator(system_prompt, human_prompt)
            out_put = chain.run(subject=subject,
                                question=question)
    return output_repraser(out_put)

def qa_agent_back(subject, question, temp=0, model_name='gpt-4-0125-preview'):
    system_prompt = (
        "You are a professional specialized in {subject}. You need to help me answer the given question."
        "Notice that you need to solve the question step by step. Do not jump to the answer directly. Do not use latex notations"
        "Your intermediate steps and thoughts are critical, you should start from the last step to the first step!" 
        "\n{format_instructions}")
    human_prompt = "{question}"
        
    response_schemas = [
        ResponseSchema(name="Chain of Thought",
                       description="Provide step by step analysis in reverse order. For instance, you should follow the pattern 'step n:... \nstep n-1:...'"),
        ResponseSchema(name="Final Answer",
                       description="Give me your original answer and your backward analysis answer")
    ]
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    success = False
    while not success:
        try:
            worker = ChatModelWorker(output_parser=output_parser,temperature=temp, model=model_name)
            chain = worker.chain_generator(system_prompt, human_prompt)
            out_put = chain.run(subject=subject,
                                question=question)

            success = True
        except:
            worker = ChatModelWorker(output_parser=output_parser,temperature=temp, model=model_name)
            chain = worker.chain_generator(system_prompt, human_prompt)
            out_put = chain.run(subject=subject,
                                question=question)
    return (out_put)


In [ ]:
subject = 'math'
question = '''
Suppose that the UW midnight sun solar car team decided, unwisely, to use an undamped
suspension system with spring constant k = 1 and dampening constant b = 0. In the absence of
a forcing term, a spring with these physical properties and the initial conditions y(0) = 1 and
y'(0) = 0 oscillates forever between y = −1 and y = 1. Find
a forcing terms so that fap(t) remains bounded while y(t) is unbounded (i.e. there is
a real number M (notice that M cannot be +-inf) such that for all t  |fap(t)| < M but limit of y(t) = ±inf when t approaching to inf).
You should explicitly compute what your fap(t) functions are.'''
result = qa_agent(subject=subject,question=question)
print(result['Chain of Thought'])
print('-------------------------------------------')
print(result['Backwards Chain of Thought'])
print('-------------------------------------------')
print(result['Final Answer'])

In [ ]:
subject = 'math'
question = '''
Suppose that the UW midnight sun solar car team decided, unwisely, to use an undamped
suspension system with spring constant k = 1 and dampening constant b = 0. In the absence of
a forcing term, a spring with these physical properties and the initial conditions y(0) = 1 and
y'(0) = 0 oscillates forever between y = −1 and y = 1. Find
a forcing terms so that fap(t) remains bounded while y(t) is unbounded (i.e. there is
a real number M (notice that M cannot be +-inf) such that for all t  |fap(t)| < M but limit of y(t) = ±inf when t approaching to inf).
You should explicitly compute what your fap(t) functions are.'''
result = qa_agent_back(subject=subject,question=question)


In [ ]:

result

# Verify Chain steps

In [10]:
def forward_agent(subject, question, temp=0, model_name='gpt-4-0125-preview'):
    system_prompt = (
        "You are a professional specialized in {subject}. You need to help me answer the given question."
        "Notice that you need to solve the question step by step. Do not jump to the answer directly."
        "Your intermediate steps and thoughts are critical!"
        "\n{format_instructions}")
    human_prompt = "{question}"
        
    response_schemas = [
        ResponseSchema(name="Chain of Thought",
                       description="Provide step by step analysis. For instance, you should follow the pattern 'step 1:... \nstep 2:...'"),
        ResponseSchema(name="Number of Steps Proposed",
                       description="return a simple integer output which indicates the number of steps you proposed in the Chain of Thought"),
        ResponseSchema(name="Final Answer",
                       description="Give me your final answer")
    ]
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    success = False
    while not success:
        try:
            worker = ChatModelWorker(output_parser=output_parser,temperature=temp, model=model_name)
            chain = worker.chain_generator(system_prompt, human_prompt)
            out_put = chain.run(subject=subject,
                                question=question)

            success = True
        except:
            worker = ChatModelWorker(output_parser=output_parser,temperature=temp, model=model_name)
            chain = worker.chain_generator(system_prompt, human_prompt)
            out_put = chain.run(subject=subject,
                                question=question)
    return out_put
def back_check_agent(subject, current_step,cot,final_answer, temp=0, model_name='gpt-4-0125-preview'):
    system_prompt = (
        '''You are a professional specialized in {subject}. You need to help me verify my steps when I solve the question.
        Your goal is help me first verify that given the final answer as {final_answer} and I am currently at step #{current_step},
        is the current step correct?. If it is correct, then verify that will my current step make the previous step hold? In other words, 
        check the logic consistency of step n and step n-1 where n is my current step. It is important that for each analysis, ignore steps
        other than the current step and the previous step!
        \n{format_instructions}
''')
    human_prompt = "Here is my complete thought process {cot}"
        
    response_schemas = [
        ResponseSchema(name="Verification",
                       description="Help me verify the correctness of the current step and the logic consistency, and tell me the reason"),
        ResponseSchema(name="Step Correctness",
                       description="say [YES] if correct, otherwise [NO]"),
        ResponseSchema(name="Logic Consistency",
                       description="say [YES] if consistent, otherwise [NO]")
    ]
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    success = False
    while not success:
        try:
            worker = ChatModelWorker(output_parser=output_parser,temperature=temp, model=model_name)
            chain = worker.chain_generator(system_prompt, human_prompt)
            out_put = chain.run(subject=subject,
                                current_step=current_step,cot=cot,final_answer=final_answer)

            success = True
        except:
            worker = ChatModelWorker(output_parser=output_parser,temperature=temp, model=model_name)
            chain = worker.chain_generator(system_prompt, human_prompt)
            out_put = chain.run(subject=subject,
                                current_step=current_step,cot=cot,final_answer=final_answer)
    return out_put

In [ ]:
# subject = 'math'
# question = '''
# Solve for x: x^2+10x+24=0
# '''
# result = forward_agent(subject=subject, question=question)
# result

In [ ]:
# forward_result = output_repraser(result)

In [ ]:
# cot,steps,final_answer = forward_result.values()

In [ ]:
# current_step = 7

In [ ]:
# back_check_result = back_check_agent(subject=subject,current_step=current_step,cot=cot,final_answer=final_answer)

In [ ]:
# output_repraser(back_check_result)

In [ ]:
# check_list = []
# for i in range(7):
#     current_step = 7-i
#     back_check_result = back_check_agent(subject=subject,current_step=current_step,cot=cot,final_answer=final_answer)
#     response = output_repraser(back_check_result)
#     print(response)
#     check_list.append(response['Final Answer'])
    

In [16]:
subject = 'math'
question = '''
Solve for y(x): y''+y'-2y=0. y(0)=1, y->0 as x-> inf
'''
result = forward_agent(subject=subject, question=question)
forward_result = output_repraser(result)
print(forward_result)
print('------------------------------------------------------')
cot,steps,final_answer = forward_result.values()
check_list = []
for i in range(int(steps)):
    current_step = int(steps)-i
    back_check_result = back_check_agent(subject=subject,current_step=current_step,cot=cot,final_answer=final_answer)
    response = output_repraser(back_check_result)
    print(f'Step {current_step}',response,'\n\n')
    check_list.append((response['Step Correctness'],response['Logic Consistency']))
print('------------------------------------------------------')
print(check_list)

{'Chain of Thought': "step 1: Identify the differential equation type. The given equation is a second-order linear homogeneous differential equation of the form y'' + p(x)y' + q(x)y = 0, where p(x) = 1 and q(x) = -2.\nstep 2: Solve the characteristic equation associated with the differential equation. The characteristic equation is r^2 + r - 2 = 0.\nstep 3: Factor the characteristic equation. It factors to (r + 2)(r - 1) = 0, giving us roots r1 = -2 and r2 = 1.\nstep 4: Write the general solution to the differential equation using the roots from the characteristic equation. Since the roots are real and distinct, the general solution is y(x) = C1*e^(r1*x) + C2*e^(r2*x) = C1*e^(-2x) + C2*e^(x).\nstep 5: Apply the initial condition y(0) = 1. Plugging in x = 0, we get 1 = C1*e^(0) + C2*e^(0) = C1 + C2. So, C1 + C2 = 1.\nstep 6: Apply the condition y -> 0 as x -> inf. For y to approach 0 as x approaches infinity, the term involving e^(x) must have a coefficient of 0 because e^(x) grows with